In [1]:
from langchain_core.documents import Document
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

/Users/shreegpersonal/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/shreegpersonal/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dir_loader = DirectoryLoader("../data/pdf_files", glob="*.pdf", loader_cls=PyMuPDFLoader)
documents = dir_loader.load()
documents

[Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-08-12T09:48:22+00:00', 'source': '../data/pdf_files/ShrivatsaSaankhya_Env_Resume.pdf', 'file_path': '../data/pdf_files/ShrivatsaSaankhya_Env_Resume.pdf', 'total_pages': 1, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-08-12T09:48:22+00:00', 'trapped': '', 'modDate': 'D:20250812094822Z', 'creationDate': 'D:20250812094822Z', 'page': 0}, page_content='Saankhya Shrivatsa\n+1 (619) 415-3843 | shrivatsaankhya.bus@gmail.com | linkedin.com/in/saankhya-shrivatsa |\nEducation\nUniversity of California San Diego\nSan Diego, CA\nBachelor of Science in Global Health; Minor in Computational Social Sciences\nSept. 2022 – December 2025\nInventure Academy\nBangalore, India\nHigh School Diploma; 5 CAIE AS/A Levels including Physics, Chemistry, Math, and Biology\nMay 2020 – June 2022\nExperience\nResearch Assistant – Climate Epidemiology\nJune 2024 – 

In [3]:
# Text Splitting

def split_documents(documents, chunk_size=500, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]

    )
    return text_splitter.split_documents(documents)

split_docs = split_documents(documents)
split_docs

[Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-08-12T09:48:22+00:00', 'source': '../data/pdf_files/ShrivatsaSaankhya_Env_Resume.pdf', 'file_path': '../data/pdf_files/ShrivatsaSaankhya_Env_Resume.pdf', 'total_pages': 1, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-08-12T09:48:22+00:00', 'trapped': '', 'modDate': 'D:20250812094822Z', 'creationDate': 'D:20250812094822Z', 'page': 0}, page_content='Saankhya Shrivatsa\n+1 (619) 415-3843 | shrivatsaankhya.bus@gmail.com | linkedin.com/in/saankhya-shrivatsa |\nEducation\nUniversity of California San Diego\nSan Diego, CA\nBachelor of Science in Global Health; Minor in Computational Social Sciences\nSept. 2022 – December 2025\nInventure Academy\nBangalore, India\nHigh School Diploma; 5 CAIE AS/A Levels including Physics, Chemistry, Math, and Biology\nMay 2020 – June 2022\nExperience\nResearch Assistant – Climate Epidemiology\nJune 2024 – 

In [4]:
import numpy as np
import os
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
class EmbeddingManager:
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        self.model_name = model_name
        self.model = None
        self._load_model() # calls the private method to load the model

    def _load_model(self):
        try:
            self.model = SentenceTransformer(self.model_name)
            print(f"Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model: {self.model_name}. Error: {e}")
            raise

    def generate_embeddings(self, texts : list[str]) -> np.ndarray:
        if not self.model:
            raise ValueError("Model not selected.")

        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"shape of embeddings {embeddings.shape}")
        return embeddings

In [6]:
embedding_manager = EmbeddingManager()
embedding_manager

Embedding dimension: 384


In [7]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

    def remove_all_documents(self):
        self.collection.delete(where={"doc_index": {"$gte": 0}})

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 18


In [8]:
vectorstore.remove_all_documents()

In [9]:
chunks = split_docs
chunks

[Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-08-12T09:48:22+00:00', 'source': '../data/pdf_files/ShrivatsaSaankhya_Env_Resume.pdf', 'file_path': '../data/pdf_files/ShrivatsaSaankhya_Env_Resume.pdf', 'total_pages': 1, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-08-12T09:48:22+00:00', 'trapped': '', 'modDate': 'D:20250812094822Z', 'creationDate': 'D:20250812094822Z', 'page': 0}, page_content='Saankhya Shrivatsa\n+1 (619) 415-3843 | shrivatsaankhya.bus@gmail.com | linkedin.com/in/saankhya-shrivatsa |\nEducation\nUniversity of California San Diego\nSan Diego, CA\nBachelor of Science in Global Health; Minor in Computational Social Sciences\nSept. 2022 – December 2025\nInventure Academy\nBangalore, India\nHigh School Diploma; 5 CAIE AS/A Levels including Physics, Chemistry, Math, and Biology\nMay 2020 – June 2022\nExperience\nResearch Assistant – Climate Epidemiology\nJune 2024 – 

In [10]:
## Converting the text to embeddings

texts = [chunk.page_content for chunk in chunks]

embeddings = embedding_manager.generate_embeddings(texts)
vectorstore.add_documents(chunks, embeddings)

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.96it/s]

shape of embeddings (18, 384)
Adding 18 documents to vector store...
Successfully added 18 documents to vector store
Total documents in collection: 18


In [11]:
embeddings[1]

array([ 1.83387790e-02,  8.08962733e-02,  2.09134053e-02,  1.14547059e-01,
        5.48869409e-02,  8.93306583e-02, -1.02011964e-01,  3.47146718e-03,
       -5.44685647e-02,  2.06589401e-02, -6.69584190e-03, -3.18160877e-02,
        4.00375985e-02,  5.52755408e-02,  2.12789811e-02, -3.06684966e-03,
       -4.07734215e-02,  1.40934892e-04, -6.36898875e-02, -1.18239434e-03,
       -3.82971242e-02,  6.46867454e-02, -5.75176552e-02,  1.82433482e-02,
       -9.14873406e-02,  5.93479630e-03, -7.05572590e-02,  3.76166925e-02,
       -2.51279920e-02,  1.00590207e-01,  1.21803246e-02,  1.79777425e-02,
       -9.75954160e-02,  3.57726179e-02,  3.59861292e-02,  6.18009418e-02,
        3.82712744e-02, -5.21687865e-02, -7.88028240e-02,  1.26000894e-02,
       -4.36004624e-02, -7.64704123e-02, -6.77130464e-03, -1.12373397e-01,
        3.62031981e-02, -5.79416454e-02,  2.20499448e-02, -2.13676244e-02,
       -4.52533364e-02, -2.46788692e-02,  7.13149533e-02,  3.79058532e-02,
        4.81487960e-02,  

In [12]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = -100.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    print(similarity_score)
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [13]:
rag_retriever.retrieve("who is shreeg?")

Retrieving documents for query: 'who is shreeg?'
Top K: 5, Score threshold: -100.0


Batches: 100%|██████████| 1/1 [00:00<00:00, 24.18it/s]

shape of embeddings (1, 384)
-0.3831629753112793
-0.6480484008789062
-0.6647789478302002
-0.7561562061309814
-0.8267606496810913
Retrieved 5 documents (after filtering)


[{'id': 'doc_db0060bf_10',
  'content': 'Shree Gopalakrishnan\n858-761-2827 | shree.hridai@gmail.com | Linkedin: shree-gk W | Github: Shree-G W\nFull Stack Engineer with a track record of taking products from idea to production. My experience ranges from building an\naward-winning IoT hardware MVP to deploying a scalable, full-stack AI analytics platform that boosted sales conversion.\nCurrently looking to join a company where I can apply my AI and full stack expertise to build end-to-end systems that',
  'metadata': {'title': '',
   'author': '',
   'source': '../data/pdf_files/Resume_Shree_G.pdf',
   'moddate': '2025-09-17T23:16:25+00:00',
   'creationDate': 'D:20250917231625Z',
   'file_path': '../data/pdf_files/Resume_Shree_G.pdf',
   'creationdate': '2025-09-17T23:16:25+00:00',
   'modDate': 'D:20250917231625Z',
   'trapped': '',
   'page': 0,
   'subject': '',
   'keywords': '',
   'format': 'PDF 1.5',
   'doc_index': 10,
   'total_pages': 1,
   'producer': 'pdfTeX-1.40.26',
   '

So far we've:

1. Loaded documents via document loaders into a list of Document objects that have the text of the documents, as well as related metadata.
2. Split up the Document texts via a text splitter into more manageable chunks.
3. We've used SentenceTransformer, which is a python class from the sentence-transformers library. We've loaded a model into the SentenceTransformer.
4. We used the SentenceTransformer to convert text chunks into embeddings. Embeddings are Numpy Arrays that are essentially very high dimensional vectors that are used for semantic search and reasoning tasks.
5. We created a vector store to store all these vectors using ChromaDB. This is a local file storage bucket, but in production we usually host in the cloud. We initialize the store and add all the documents with the vector embeddings that we computed in step 4.
6. We then created a RAGRetriever, that if given a query, conerts the query into a vector using the same generate_embeddings function, and then computes similarity search across the entire vector-db for semantically similar chunks.
    a. side note: this is where I'm running into errors. Not sure if my vector-db initial document store is too tiny, as it is just two resume PDFs. Another possible error is that the pdf might be formatted weirdly, or chunking might be splitting up things weirdly as well. Yet another possible error is that the similarity search engine I'm using may be non-optimized. IDK!

What we have yet to do:
7. Connect to an LLM, give it context based on the similarity search results as well as the user prompt, and then let it run.

In [18]:
# Simple RAG Pipeline w/ Groq
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

# Initialize GROQ LLM
groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.3-70b-versatile",temperature=0.1,max_tokens=1024)

# Simple RAG function: retrieve + generate
def rag_simple(query,retriever,llm,top_k=5):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely. Cite your answers.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

In [19]:
answer=rag_simple("what makes saankhya and shree a good fit romantically",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'what makes saankhya and shree a good fit romantically'
Top K: 5, Score threshold: -100.0


Batches: 100%|██████████| 1/1 [00:02<00:00,  2.61s/it]


shape of embeddings (1, 384)
-0.4282459020614624
-0.7770044803619385
-0.781403660774231
-0.8098413944244385
-0.8159867525100708
Retrieved 5 documents (after filtering)
Based on the provided context, it's challenging to determine a romantic fit between Saankhya and Shree, as the information is primarily professional. However, some potential commonalities can be inferred:

1. **Shared interests in technology**: Both Saankhya and Shree have experience with programming languages, frameworks, and tools, which could create a foundation for interesting conversations and shared hobbies [1].
2. **Similar educational background**: Saankhya is currently pursuing a degree, and Shree's educational background is not explicitly mentioned, but they both seem to value knowledge and personal growth [1].
3. **Health and wellness**: Saankhya's involvement with Student Health Advocates (SHA) and Shree's experience with health-related projects (e.g., IoT hardware MVP) might indicate a mutual interest in hea

In [16]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("Shree's best accomplishments", rag_retriever, llm, top_k=10, min_score=-10, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'Shree's best accomplishments'
Top K: 10, Score threshold: -10


Batches: 100%|██████████| 1/1 [00:00<00:00, 25.07it/s]

shape of embeddings (1, 384)
-0.37240028381347656
-0.7041860818862915
-0.7445937395095825
-0.7700724601745605
-0.7792203426361084
-0.7817902565002441
-0.8025708198547363
-0.8163086175918579
-0.818108081817627
-0.8604607582092285
Retrieved 10 documents (after filtering)


Answer: Based on the provided context, Shree's best accomplishments include:

1. Building a weekly call volume forecaster at ServiceMob, achieving 9% MAPE to improve staffing decisions for 500+ support agents.
2. Deploying a forecasting API inside a consumer-side web app accessed in real-time by 5+ call-center agencies at ServiceMob.
3. Developing a KNN-based resume classifier at Point Perfect Technology Solutions, classifying over 450+ resumes into best-fit jobs.
4. Building a scalable data pipeline to process 200,000+ dealer records from a third-party API into Google BigQuery at iFrog Marketing Solutions.
5. Designing a full-stack AI dashboard to provide real-time customer segmentation and lead scoring for 67 dealerships at iFrog Marketing Solutions, resulting in a 15% increase in ad sales conversion rates.
Sources: [{'source': '../data/pdf_files/Resume_Shree_G.pdf', 'page': 0, 'score': -0.37240028381347656, 'preview': 'Shree Gopalakrishnan\n858-761-2827 | shree.hridai@gmail.com | Li

In [17]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("what technologies is shree most comfortable with", top_k=6, min_score=-10, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'what technologies is shree most comfortable with'
Top K: 6, Score threshold: -10


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.46it/s]

shape of embeddings (1, 384)
-0.17604172229766846
-0.19512581825256348
-0.4525928497314453
-0.4766504764556885
-0.6119284629821777
-0.6181368827819824
Retrieved 6 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
Shree Gopal

akrishnan
858-761-2827 | shree.hridai@gmail.com | Linkedin: shree-gk W | Github: Shree-G W
Full Stack Engineer with a track record of taking products from idea to production. My experience ranges from building an
award-winning IoT hardware MVP to deploying a scalable, full-stack AI analytics platform that boosted sales conversion.
Currently looking to join a company where I can apply my AI and full stack expertise to build end-to-end systems that

solve complex challenges and ship features that actually help people.
Technical Skills
Languages & Frameworks: JavaScript (ES6+), TypeScript, Python, SQL, React.js, Next.js, FastAPI, Node.js, Prisma
Frontend Tools: Tailwind CSS, ShadCN, Styled Components, HTML, Framer Motion, D3
Backend & Data: Google BigQuery, PostgreSQL, Redis, RESTful API, ETL / ETL Processes, Docker, Websockets
AI/Data Science: Scikit-learn, Pandas, NumPy, K-Means Clustering, OpenAI API, Hugging Face

• Designed a responsive UI with Tailwind CSS for consistent styling and